In [21]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

In [22]:
class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFKC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(files=paths, trainer=trainer)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)

In [23]:
file1 = open('Arabic_corpus.txt', 'r',encoding = 'utf-8')
all_text = file1.readlines()
print (all_text[1:3])

['طرحت المطرب المغربيه 4 اغنيه وطنيه جديده بعنوان المسيره من كلماتها والحانها وتوزيع الفنان 4 8 الاغنيه الجديده 4 جاءت في سياق احتفال 2 بالذكري الاربعين لحدث 5 الخضراء واسترجاع الاقاليم الصحراويه 1 ما جعل وزاره الاتصال تختارها لتكون ضمن تشكيله الاغاني الوطنيه الجديده التي خلدت الحدث وفي سياق متصل من المرتقب ان تشارك 4 في حفل غنائي بالعيون بمناسبه الذكري نفسها يوم سادس نونبر الجاري والتي ستحييها مجموعه من الاسماء الفنيه المغربيه والاجنبيه وجدير بالاشاره الي ان المطربه 4 سبق لها ان شاركت في العديد من السهرات داخل 2 وخارجه وتختص في اداء الاغاني الطربيه كما لها العديد من الاعمال الفنيه الخاصه بها منها البوم عمري لك 4 8 تعليق 4 خاص 4 8 يحتفي بالفنان في سهره 5 9 حل الفنان 4 8 ضيفا علي الاذاعه الوطنيه يوم الخميس الماضي لتسجيل حلقه خاصه به من برنامج هذي ليلتنا من اعداد وتقديم 4 8 الذي اختار تكريمه اعترافا بعطاءاته وابداعاته المتواصله وسيكون مستمعو الاذاعه الوطنيه علي موعد مع الفقرات المتنوعه للسهره التكريميه من اخراج 4 8 والتي ستبث ليله راس السنه الميلاديه علي اثير الاذاعه الوطنيه علي الساعه ا

In [24]:
paths = ['Arabic_corpus.txt']
print (paths)

tokenizer = BPE_token()
# train the tokenizer model
tokenizer.bpe_train(paths)

# saving the tokenized data in our specified folder 
save_path = './Arabic_bot/'
tokenizer.save_tokenizer(save_path)


['Arabic_corpus.txt']


In [25]:
import random

n = int(len(all_text)/10)

random.shuffle(all_text)

test_data = all_text[:n]
train_data = all_text[n:]

print (len(train_data))

print (len(test_data))

# printing n elements from list


5242
582


In [26]:
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
model = TFGPT2LMHeadModel(config)

In [27]:
print (model.num_parameters)

<bound method TFModelUtilsMixin.num_parameters of <transformers.models.gpt2.modeling_tf_gpt2.TFGPT2LMHeadModel object at 0x000001C118AC3EE0>>


In [28]:
print('Words in vocabulary: ', tokenizer.vocab_size)

Words in vocabulary:  50000


In [59]:
single_string = ''
for file in train_data:
    single_string += file + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [60]:
len(string_tokenized)

67964858

In [52]:
import tensorflow as tf

In [56]:
examples = []
block_size = 10
BATCH_SIZE = 8
BUFFER_SIZE = 10
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
    examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [57]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric],run_eagerly=True)

In [58]:
num_epoch = 100
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/100


ResourceExhaustedError: Exception encountered when calling layer "wte" "                 f"(type TFSharedEmbeddings).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[8,9,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResourceGather]

Call arguments received by layer "wte" "                 f"(type TFSharedEmbeddings):
  • inputs=tf.Tensor(shape=(8, 9), dtype=int64)
  • mode='embedding'

In [ ]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
output_dir = './Arabic_bot/'
# creating directory if it is not present
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

model_to_save = model.module if hasattr(model, 'module') else model

output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
model.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained(output_dir)

In [ ]:
with open ('./Arabic_bot/test_original_corpus.txt', 'w',encoding='utf-8') as f:
    for line in test_data:
        f.write(f"{line}\n")

In [ ]:
import glob
import tqdm

In [ ]:
def generate_Text_bot(test_data,bot_path):
    
    output_file = './Arabic_bot/test_bot_corpus.txt'
    tokenizer = GPT2Tokenizer.from_pretrained(bot_path)
    model = TFGPT2LMHeadModel.from_pretrained(bot_path)
    for line in tqdm.tqdm(test_data):
        start_token = ' '.join(line.split()[:2])
        print (start_token)
        input_ids = tokenizer.encode(start_token, return_tensors='tf')
        # getting out output
        beam_output = model.generate(
          input_ids,
          max_length = len(line.split()),
          num_beams = 5,
          temperature = 0.7,
          no_repeat_ngram_size=2,
          num_return_sequences=5
        )
        output_text = tokenizer.decode(beam_output[0],skip_special_tokens = True)
        print (output_text)
        with open(output_file, 'a',encoding='utf-8') as f:
            f.write(f"{output_text}\n")
            
        #break

In [ ]:
generate_Text_bot(test_data,'./Arabic_bot/')

# Pretrained Hugging face

In [61]:
from transformers import GPT2TokenizerFast, pipeline
from transformers import GPT2LMHeadModel

In [63]:
!pip install arabert

     ---------------------------------------- 0.0/179.3 kB ? eta -:--:--
     -- ------------------------------------- 10.2/179.3 kB ? eta -:--:--
     ------ ------------------------------ 30.7/179.3 kB 435.7 kB/s eta 0:00:01
     ------ ------------------------------ 30.7/179.3 kB 435.7 kB/s eta 0:00:01
     ------ ------------------------------ 30.7/179.3 kB 435.7 kB/s eta 0:00:01
     -------- ---------------------------- 41.0/179.3 kB 151.3 kB/s eta 0:00:01
     ------------ ------------------------ 61.4/179.3 kB 204.8 kB/s eta 0:00:01
     ------------------- ----------------- 92.2/179.3 kB 275.8 kB/s eta 0:00:01
     ---------------------- ------------- 112.6/179.3 kB 328.2 kB/s eta 0:00:01
     ---------------------- ------------- 112.6/179.3 kB 328.2 kB/s eta 0:00:01
     ---------------------------- ------- 143.4/179.3 kB 304.6 kB/s eta 0:00:01
     ---------------------------------- - 174.1/179.3 kB 338.5 kB/s eta 0:00:01
     ------------------------------------ 179.3/179.3

In [64]:
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel

In [65]:
from arabert.preprocess import ArabertPreprocessor

In [66]:
MODEL_NAME='aubmindlab/aragpt2-base'

In [67]:
arabert_prep = ArabertPreprocessor(model_name=MODEL_NAME)

In [ ]:
file1 = open('Arabic_corpus.txt', 'r',encoding = 'utf-8')
all_text = file1.readlines()
print (all_text[1:3])

In [68]:
import random

n = int(len(all_text)/10)

random.shuffle(all_text)

test_data = all_text[:n]
train_data = all_text[n:]

print (len(train_data))

print (len(test_data))

# printing n elements from list


5242
582


In [74]:
print(test_data[0])

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 9 9 المزمع اقامتها في الفتره ما بين يوليوز والخامس من غشت القادمين ومثل 3 7 في مراسم توقيع العقود الامير 4 8 8 رئيس الاتحاد فيما مثل الاتحاد المصري المهندس 4 8 8 رئيس الاتحاد ويحصل الفريق الفائز بلقب البطوله علي عشره ملايين 5 سعودي في الوقت الذي لم يتم الكشف فيه عن باقي الجوائز ولكن تردد ان كل فريق مشارك سيحصل علي الف 5 وواجهت البطوله العربيه صعوبات ماليه بالغه مما اثر علي استقرار البطوله حيث جرت النسخه الاخيره من دوري ابطال العرب في قبل ان تعود البطوله تحت مسمي جديد هو كاس 9 9 في موسم لكنها توقفت مره اخري اضحت مباريات الكلاسيكو الاسباني بين 3 7 3 تحمل اكثر من البعد الرياضي في 2 حيث ان نسبه من المساجين المتواجدين في سجن مغربي يدعي سجن 3 2 تسبب في سجنهم كل من 4 8 4 8 نجمي 3 7 3 7 واشار تقرير نشرته الصحيفه صحيفه 3 الكتالونيه الي ان النسبه المذكوره من المساجين اعتقلت جراء تورطها في شجارات وجرائم بسبب الكلاسيكو والنقاش الحاد عن 4 4 خاصه في مقاهي المدينه التي تتحول لشبه مدرجات الملعب خلال لقاءات 3 3 وذلك بتوافد المئات من الزبائن عليها لمتابعه الكلاسيكو اخبارنا 

In [75]:
start_token = ' '.join(test_data[0].split()[:2])
print (start_token)

1 1


In [71]:
text=test_data[0]
text_clean = arabert_prep.preprocess(text)



In [73]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)
generation_pipeline = pipeline("text-generation",model=model,tokenizer=tokenizer)


Downloading:   0%|          | 0.00/553M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/aragpt2-base were not used when initializing GPT2LMHeadModel: ['ln_f.bias', 'ln_f.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at aubmindlab/aragpt2-base and are newly initialized: ['emb_norm.bias', 'emb_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

In [83]:
start_token ="وابلغ المصدر المجهول الصحيفه ان"
generation_pipeline(start_token,
    pad_token_id=tokenizer.eos_token_id,
    num_beams=10,
    max_length=20,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)[0]['generated_text']

'وابلغ المصدر المجهول الصحيفه ان ، من و وط في " أو. ولا …1 وب'

# Finding the graph 

In [85]:
import numpy as np

In [131]:
At_VECTORIZED_CORPUS = "./Arabic_vectorized_corpus_sample1%_CBOW_1gram_6.npy"
sample = np.load(At_VECTORIZED_CORPUS, allow_pickle=True)
sample = sample.tolist()

In [132]:
sample[0]


{'document_index': 2210,
 'vectorized_text': {'1': array([-2.22429752, -5.24652338, 10.73093414,  6.8847518 , -0.62628794,
          6.78570366]),
  'نقابه': array([-0.63795984,  1.25795734,  4.03692675,  0.29276645, -0.26610079,
         -0.11858656]),
  'عمال': array([0.71085626, 0.69991124, 3.1185813 , 1.76478994, 1.4660722 ,
         2.67626166]),
  'الانعاش': array([ 2.291682  , -0.53498554,  5.90126801,  3.5009923 ,  2.74611211,
          1.33218062]),
  'الوطني': array([-4.11060095, -4.75963974,  0.36291021, -0.16075151, -1.22796738,
         -4.52696037]),
  'التابعه': array([ 7.85348177, -2.61570406, 13.50592804,  1.28739142, -0.47856274,
         -1.48942971]),
  '3': array([-2.7894206 , -6.66022873,  2.43006873, -0.14766884, -2.95838356,
         -5.57351494]),
  '7': array([-2.45979834, -6.68042994,  3.89928865,  0.02736536, -3.93241763,
         -5.83295631]),
  'في': array([-0.3575978 , -2.30077553, -1.31894565, -1.80745256,  0.47699654,
         -0.31612265]),
  'بيان': 

In [133]:
documents_number = []
for text in sample:
    documents_number.append(text['document_index'])
print (documents_number)

[2210, 2759, 2615, 955, 1283, 2670, 2304, 2228, 119, 2380, 1810, 2834, 1076, 1177, 2591, 291, 2661, 946, 816, 718, 125, 486, 2340, 700, 576, 1497, 683, 861, 1536]


In [134]:
file1 = open('Arabic_corpus.txt', 'r',encoding = 'utf-8')
#all_text = file1.readlines()
test_data = []
train_data = []
for index , line in enumerate(file1.readlines()):
    if index in documents_number:
        test_data.append(line)
    else:
        train_data.append(line)

In [135]:
len(test_data)

29

In [136]:
with open ('./Arabic_bot/test_original_corpus_CBOW.txt', 'w',encoding='utf-8') as f:
    for line in test_data:
        f.write(f"{line}")